<a href="https://colab.research.google.com/github/inooni/COSE362/blob/main/cose362.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;

16 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-12-12 01:34:13--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?PIwsaAhtAIg2lFuHCfvi8alMt-1wQzMRUYhpTb4rCWLywOinMDhWXtMoTEr0q9Gn6HuqvDO3nielp97G6nW_FJMyxOpnpQF8YDN6Ksgnt6P8ZYh4PoLzZRd48uXmbXuqcrF_nz246AchlwRQYX7OY95snVfyhec7VfAffVH9WpxCWf3Pyo3LE6FMZ1WxBQv7oZdAktAEO9c3LiqZc15E_1puoQ [following]
--2019-12-12 01:34:14--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?PIwsaAhtAIg2lFuHCfvi8a

# 새 섹션

In [ ]:

!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-8-0' for regex 'cuda-8.0'
Note, selecting 'libcuda-8.0-1' for regex 'cuda-8.0'
cuda-8-0 is already the newest version (8.0.61-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-y05wxkmz
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-y05wxkmz
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=9bbea4580e41cdbbb0c644237903f1bb7713f7da7e9759fbf9cafab535bd7ce1
  Stored in directory: /tmp/pip-ephem-wheel-cache-n858y6s3/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <cstdio>
#include<ctime>
using namespace std;


__device__ void warpReduce(volatile float* sdata, int tid) {
sdata[tid] += sdata[tid + 32];
sdata[tid] += sdata[tid + 16];
sdata[tid] += sdata[tid + 8];
sdata[tid] += sdata[tid + 4];
sdata[tid] += sdata[tid + 2];
sdata[tid] += sdata[tid + 1];
}

__global__
void Fdot5(float* conv_input, float* conv_filter, float* conv_result, int i, float bias){
  __shared__ float sdata[1024];
  float init=0.0;
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int x=(bid/81)%81;
  unsigned int y=bid%81;
  int a,b,c;
  __syncthreads();	
  if(tid<363){
  __syncthreads();	
    c=tid%3;
  __syncthreads();	
    b=(tid/3)%11;
  __syncthreads();	
    a=(tid/33)%11;
  __syncthreads();	
    sdata[tid]=conv_input[336*3*(4*x + a)+3*(4*y + b)+c] * conv_filter[11*3*96*a+3*96*b+96*c+i];
  __syncthreads();	
  }
  else sdata[tid]=0;
  __syncthreads();	
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+bias>0) conv_result[83*96*x+96*y+i] = sdata[0]+bias;
      else conv_result[83*96*x+96*y+i]=0.0;
  }
  __syncthreads();
  
}


int main() {
  clock_t start,end;
  float result=0;
  float *A;
  float *B;
  float *C;
  float *D;
  cudaMallocManaged(&A, sizeof(float)*110000000);
  cudaMallocManaged(&B, sizeof(float)*110000000);
  cudaMallocManaged(&C,sizeof(float)*110000000);
  cudaMallocManaged(&D,sizeof(float)*110000000);
  
  for(int i=0; i<110000000; i++){
    A[i]=0.1;
  }
  
  for(int i=0; i<110000000; i++) {
     B[i]= 0.1;
  }
  
  for(int i=0; i<110000000; i++){
      C[i]=0.0;
  }
for(int i=0; i<110000000; i++){
      D[i]=0.0;
  }
  printf("hello\n");
  start=clock();
  for (int i = 0; i < 96; i++) {
      Fdot5<<<6561, 512>>>(A, B, C, i, 0);
    	cudaDeviceSynchronize();
 	}
  
 	end=clock();

  cudaError_t rc = cudaGetLastError();
  if (rc != cudaSuccess) printf("Last CUDA error %s\n", cudaGetErrorString(rc));

  printf("%f\n",((float)(end-start))/1000000);
 printf("%f\n",C[0]);
  printf("asf\n");
}

hello
0.035707
3.630000
asf



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%cu
#include <stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
#include<algorithm>
using namespace std;

#define HEADER 54
#define WIDTH 336
#define HEIGHT 336
#define CAT1 2192
#define CAT2 1800
#define CAT3 0
#define CAT4 1448
#define CAT5 1688
#define CAT6 1136

#define CAT1_test 20
#define CAT2_test 20
#define CAT3_test 0
#define CAT4_test 20
#define CAT5_test 20
#define CAT6_test 20


using namespace std;

typedef struct poolmem{
   int p,q,r;
}poolmem;

float* conv1_input;
float* conv1_filter;
float* conv1_bias;
float* conv1_result;

float* conv2_input;
float* conv2_filter;
float* conv2_bias;
float* conv2_result; //pooling에서 오류 대비

float* conv3_input;
float* conv3_filter;
float* conv3_bias;
float* conv3_result;
//no pooling
//float conv4_input[20*20*384];
float* conv4_filter;
float* conv4_bias;
float* conv4_result;
//no pooling
//float conv5_input[20*20*384];
float* conv5_filter;
float* conv5_bias;
float* conv5_result;

poolmem conv5_maxpool[10*10*256];
poolmem conv2_maxpool[20*20*256];
poolmem conv1_maxpool[40*40*96];
int testresult[6];
int catresultcat[6];
int onlycat[6];

float* conv6_input;
float* conv6_filter;
float* conv6_bias;

float* fc1;
float* w1;
float* fc1_bias;

float* fc2;
float* w2;
float* fc2_bias;

float* fc3;

float* fc3_g;
float* fc2_g;
float* fc1_g;
float* conv6_input_g;

float* conv5_result_g;
float* conv5_input_g;

float* conv4_result_g;
float* conv4_input_g;

float* conv3_result_g;
float* conv3_input_g;

float* conv2_result_g;
float* conv2_input_g;

float* conv1_result_g;
//float conv1_input_g[40][40][96];





FILE * fp;



float LR=0.000015;
int epoch=12000;
int result;

int datasize(int t){
   if(t==1) return CAT1;
   if(t==2) return CAT2;
   if(t==3) return CAT3;
   if(t==4) return CAT4;
   if(t==5) return CAT5;
   if(t==6) return CAT6;
   return 0;
}

void getimage(int cat, int num){
    char filename[100];
    if(cat<=2) sprintf(filename,"/content/gdrive/Shared drives/COSE362/dataset/e/CAT%d/cat%d_(%d).bmp",cat,cat,num);
    else sprintf(filename,"/content/gdrive/Shared drives/COSE362/dataset/e/CAT%d/CAT%d (%d).bmp",cat,cat,num);
    FILE *fp1 = fopen(filename,"rb");
    unsigned char tmp;
    for(int i=0; i<HEADER; i++) tmp=getc(fp1);
    for(int i=0; i<HEIGHT; i++)
        for(int j=0; j<WIDTH; j++)
        {
            conv1_input[336*3*i+3*j+2]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+1]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+0]=((float)getc(fp1))/256;
        }
   fclose(fp1);
}

void gettestimage(int cat, int num){
    char filename[100];
    if(cat<=2) sprintf(filename,"/content/gdrive/Shared drives/COSE362/testset/CAT%d/cat%d (%d).bmp",cat,cat,num);
   else sprintf(filename,"/content/gdrive/Shared drives/COSE362/testset/CAT%d/CAT%d (%d).bmp",cat,cat,num);
    FILE *fp1 = fopen(filename,"rb");
    unsigned char tmp;
    for(int i=0; i<HEADER; i++) tmp=getc(fp1);
    for(int i=0; i<HEIGHT; i++)
        for(int j=0; j<WIDTH; j++)
        {
            conv1_input[336*3*i+3*j+2]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+1]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+0]=((float)getc(fp1))/256;
        }
   fclose(fp1);
}

float sum;

int testresult1[6];
int catresultcat1[6];
int onlycat1[6];

void checkanswer1(int ans){
   int tmp=0;
   printf("\n");
   for(int i=0; i<6; i++){
      printf("%f ",fc3[i]/sum);
      if(fc3[tmp]<=fc3[i]) {
         tmp=i;
      }
   }
   printf("\nanswer: %d\nresult: %d\n",ans,tmp);
   if(tmp==ans){
      testresult1[ans]++;
   }
   if(ans<5 && tmp<5){
      catresultcat1[ans]++;
   }
   if(ans<5){
      int tmp=0;
      for(int i=0; i<5; i++){
         if(fc3[tmp]<=fc3[i]) {
            tmp=i;
         }
      }
      if(tmp==ans) onlycat1[ans]++;
   }
}
void checkanswer(int ans){
   int tmp=0;
   printf("\n");
   for(int i=0; i<6; i++){
      printf("%f ",fc3[i]/sum);
      if(fc3[tmp]<=fc3[i]) {
         tmp=i;
      }
   }
   printf("\nanswer: %d\nresult: %d\n",ans,tmp);
   if(tmp==ans){
      testresult[ans]++;
   }
   if(ans<5 && tmp<5){
      catresultcat[ans]++;
   }
   if(ans<5){
      int tmp=0;
      for(int i=0; i<5; i++){
         if(fc3[tmp]<=fc3[i]) {
            tmp=i;
         }
      }
      if(tmp==ans) onlycat[ans]++;
   }
}

float randrange(float t){
    return (float)(2.0*t*((((double)rand())/2147483647.0)-0.5));
}

void initdata(int mode){
   for(int i=0; i<96; i++) conv1_bias[i]=0.0;
   for(int i=0; i<256; i++) conv2_bias[i]=0.0;
   for(int i=0; i<384; i++) conv3_bias[i]=0.0;
   for(int i=0; i<384; i++) conv4_bias[i]=0.0;
   for(int i=0; i<256; i++) conv5_bias[i]=0.0;
   for(int i=0; i<4096; i++) conv6_bias[i]=0.0;
   for(int i=0; i<4096; i++) fc1_bias[i]=0.0;
   for(int i=0; i<6; i++) fc2_bias[i]=0.0;
    
    if(mode==0){
        FILE* fp = fopen("/content/gdrive/Shared drives/COSE362/testset/init.txt","w");
        for(int i=0; i<11; i++){
            for(int j=0; j<11; j++){
                for(int k=0; k<3; k++){
                    for(int t=0; t<96; t++) fprintf(fp,"%f\n",conv1_filter[11*3*96*i+3*96*j+96*k+t]=randrange(sqrt(((float)6.0)/363)));
                }
            }
        }
        
        for(int i=0; i<5; i++){
            for(int j=0; j<5; j++){
                for(int k=0; k<96; k++){
                    for(int t=0; t<256; t++) fprintf(fp,"%f\n",conv2_filter[5*96*256*i+96*256*j+256*k+t]=randrange(sqrt(((float)6.0)/2400)));
                }
            }
        }
        
        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<384; t++) fprintf(fp,"%f\n",conv3_filter[3*256*384*i+256*384*j+384*k+t]=randrange(sqrt(((float)6.0)/2304)));
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<384; t++) fprintf(fp,"%f\n",conv4_filter[3*384*384*i+384*384*j+384*k+t]=randrange(sqrt(((float)6.0)/3456)));
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<256; t++) fprintf(fp,"%f\n",conv5_filter[3*384*256*i+384*256*j+256*k+t]=randrange(sqrt(((float)6.0)/3456)));
                }
            }
        }

        for(int i=0; i<10; i++){
            for(int j=0; j<10; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<4096; t++) fprintf(fp,"%f\n",conv6_filter[10*256*4096*i+256*4096*j+4096*k+t]=randrange(sqrt(((float)6.0)/25600)));
                }
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<4096; j++){
                fprintf(fp,"%f\n",w1[4096*i+j]=randrange(sqrt(((float)6.0)/4096)));
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<6; j++){
                fprintf(fp,"%f\n",w2[6*i+j]=randrange(sqrt(((float)6.0)/4096)));
            }
        }
        fclose(fp);
    }
    else if(mode==1){
        FILE* fp = fopen("/content/gdrive/Shared drives/COSE362/testset/init.txt","r");
        for(int i=0; i<11; i++){
            for(int j=0; j<11; j++){
                for(int k=0; k<3; k++){
                    for(int t=0; t<96; t++) fscanf(fp,"%f",&conv1_filter[11*3*96*i+3*96*j+96*k+t]);
                }
            }
        }
        
        for(int i=0; i<5; i++){
            for(int j=0; j<5; j++){
                for(int k=0; k<96; k++){
                    for(int t=0; t<256; t++) fscanf(fp,"%f",&conv2_filter[5*96*256*i+96*256*j+256*k+t]);
                }
            }
        }
        
        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<384; t++) fscanf(fp,"%f",&conv3_filter[3*256*384*i+256*384*j+384*k+t]);
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<384; t++) fscanf(fp,"%f",&conv4_filter[3*384*384*i+384*384*j+384*k+t]);
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<256; t++) fscanf(fp,"%f",&conv5_filter[3*384*256*i+384*256*j+256*k+t]);
                }
            }
        }

        for(int i=0; i<10; i++){
            for(int j=0; j<10; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<4096; t++) fscanf(fp,"%f",&conv6_filter[10*256*4096*i+256*4096*j+4096*k+t]);
                }
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<4096; j++){
                fscanf(fp,"%f",&w1[4096*i+j]);
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<6; j++){
                fscanf(fp,"%f",&w2[6*i+j]);
            }
        }
        fclose(fp);
    }
}


__device__ void warpReduce(volatile float* sdata, int tid) {
sdata[tid] += sdata[tid + 32];
sdata[tid] += sdata[tid + 16];
sdata[tid] += sdata[tid + 8];
sdata[tid] += sdata[tid + 4];
sdata[tid] += sdata[tid + 2];
sdata[tid] += sdata[tid + 1];
}

__global__
void Fdot(float* conv_filter, float* conv_result_g, float* conv_input_g, int i, int I, int J, int K, int A, int B, int C){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  
   unsigned int j = (blockIdx.x/K)%J;
   unsigned int k = blockIdx.x%K;
  sdata[tid]=0.0;
  float init=0.0;
   for(int a1=-(A/2); a1<=(A/2); a1++){
      for(int b1=-(B/2); b1<=(B/2); b1++){
           __syncthreads();
          if((i+a1)>=0 && (i+a1)<I && (j+b1)>=0 && (j+b1)<J) init += conv_filter[B*K*C*((A/2)-a1)+K*C*((B/2)-b1)+C*k+tid]*conv_result_g[J*C*(i+a1)+C*(j+b1)+tid];
         __syncthreads();
      }
   }
  
    __syncthreads();
    sdata[tid] = init;
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
    __syncthreads();
  
  __syncthreads();
  if(tid==0) conv_input_g[i*J*K+blockIdx.x] = sdata[tid];
  __syncthreads();
}

__global__
void Fdot1(float* conv_result_g, float* conv_input, float* conv_filter, int a, int b, int t, int I, int J, int K, int A, int B, int C, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i = tid/J;
  unsigned int j = tid%J;
  unsigned int c = (bid)%C;
  unsigned int k = (bid)/C;
  
    __syncthreads();
    sdata[tid] = 0.0;
    __syncthreads();
    if(tid<800 && (i+a)>=0 && (i+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] += conv_result_g[J*C*i+C*j+c]*conv_input[J*K*(i+a)+K*(j+b)+k];
    __syncthreads();
    if(tid<800 && (i+20+a)>=0 && (i+20+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] += conv_result_g[J*C*(i+20)+C*j+c]*conv_input[J*K*(i+a+20)+K*(j+b)+k];
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>32; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
   __syncthreads();
 if (tid < 32) warpReduce(sdata, tid);
  __syncthreads();
  if(tid==0) conv_filter[B*K*C*((A/2)+a)+K*C*((B/2)+b)+C*k+c] -= sdata[0]*LR;
  __syncthreads();
}

__global__
void Fdot11(float* conv_result_g, float* conv_input, float* conv_filter, int a, int b, int t, int I, int J, int K, int A, int B, int C, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i = tid/J;
  unsigned int j = tid%J;
  unsigned int c = bid%C;
  unsigned int k = bid/C + t*(K/8);
  
    __syncthreads();
    if(tid<400 && (i+a)>=0 && (i+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] = conv_result_g[J*C*i+C*j+c]*conv_input[J*K*(i+a)+K*(j+b)+k];
    else sdata[tid] = 0.0;
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>32; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
    __syncthreads();
  if (tid < 32) warpReduce(sdata, tid);
  __syncthreads();
  if(tid==0) conv_filter[B*K*C*((A/2)+a)+K*C*((B/2)+b)+C*k+c] -= sdata[0]*LR;
  __syncthreads();
}

__global__
void Fdot2(float* fc_g, float* conv_input, float* conv_filter, float LR){
  __shared__ float tmp;
   unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();   
  if(tid==0) tmp=conv_input[bid];
  __syncthreads();   
  for(int t=0; t<4096; t+=1024){
      __syncthreads();
        conv_filter[4096*bid+tid+t] -= LR*tmp*fc_g[tid+t];
       __syncthreads();
  }
}

__global__
void Fdot3(float* fc_g, float* conv_input_g, float* conv_filter, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();   
  sdata[tid] = conv_filter[4096*bid+tid]*fc_g[tid] + conv_filter[4096*bid+tid+1024]*fc_g[tid+1024] + conv_filter[4096*bid+tid+2048]*fc_g[tid+2048] + conv_filter[4096*bid+tid+3072]*fc_g[tid+3072];
  __syncthreads();   
  
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
  __syncthreads();
  if(tid==0) conv_input_g[bid] = sdata[0];
  __syncthreads();
  
}

__global__
void Fdot4(float* conv_result_g, float* conv_input, float* conv_filter, float LR){
  __shared__ float sdata[1024];
  float init=0.0;
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int a=(bid/(3*96*11))%11;
  unsigned int b=(bid/(3*96))%11;
  unsigned int k=(bid/96)%3;
  unsigned int c=bid%96;
  int i;
  int j;
  __syncthreads();   
  for(int t=0; t<7; t++){
      __syncthreads();   
      if(tid+1024*t<6561) {
      __syncthreads();   
      i=((tid+1024*t)/81)%81;
      __syncthreads();   
      j=(tid+1024*t)%81;
      __syncthreads();   
      init += conv_input[336*3*(4*i+a)+3*(4*j+b)+k]*conv_result_g[81*96*i+96*j+c];
      __syncthreads();   
    }
    __syncthreads();   
   }
  __syncthreads();
  sdata[tid] = init;   
  __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();
  if(tid==0) conv_filter[bid] -= sdata[0]*LR;
  __syncthreads();
  
}

__global__
void Fdot5(float* conv_input, float* conv_filter, float* conv_result, int i, float bias){
  __shared__ float sdata[1024];
  float init=0.0;
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int x=(bid/81)%81;
  unsigned int y=bid%81;
  int a,b,c;
  __syncthreads();   
  if(tid<363){
  __syncthreads();   
    c=tid%3;
  __syncthreads();   
    b=(tid/3)%11;
  __syncthreads();   
    a=(tid/33)%11;
  __syncthreads();   
    sdata[tid]=conv_input[336*3*(4*x + a)+3*(4*y + b)+c] * conv_filter[11*3*96*a+3*96*b+96*c+i];
  __syncthreads();   
  }
  else sdata[tid]=0;
  __syncthreads();   
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+bias>0) conv_result[83*96*x+96*y+i] = sdata[0]+bias;
      else conv_result[83*96*x+96*y+i]=0.0;
  }
  __syncthreads();
  
}


__global__
void Fdot6(float* conv_input, float* conv_filter, float* conv_result, float *conv_bias, int x, int X, int Y, int I, int A, int B, int C, int extra){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i=bid%I;
  unsigned int y=(bid/I)%Y;
  float init;
  
  if(tid<C){
    init=0.0;
    __syncthreads();   
    for(int a=-(A/2); a<=A/2; a++){
        __syncthreads();   
        for(int b=-(B/2); b<=B/2; b++){
            __syncthreads();   
            if((x+a)>=0 && (x+a)<X && (y+b)>=0 && (y+b)<Y) init += conv_input[Y*C*(x + a)+C*(y + b)+tid] * conv_filter[A*C*I*(a+(A/2))+C*I*(b+(B/2))+I*tid+i];
             __syncthreads();   
        }
      __syncthreads();   
      }
    sdata[tid]=init;
    __syncthreads();   
  }
  else sdata[tid]=0;
  __syncthreads();   
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+conv_bias[i]>0) conv_result[(X+extra)*I*x+I*y+i] = sdata[0]+conv_bias[i];
      else conv_result[(X+extra)*I*x+I*y+i]=0.0;
  }
  __syncthreads();
  
}

__global__
void Fdot7(float* conv_input, float* conv_filter, float* conv_result, float *conv_bias){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  float init=0.0;
  __syncthreads();
      for(int a=0; a<10; a++){
      __syncthreads();
      for(int b=0; b<10; b++){
          __syncthreads();
          init += conv_input[10*256*a+256*b+tid] * conv_filter[10*256*4096*a+256*4096*b+4096*tid+bid];
         __syncthreads();
      }
      __syncthreads();
  }
    __syncthreads();
  sdata[tid]=init;
    __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+conv_bias[bid]>0) conv_result[bid] = sdata[0]+conv_bias[bid];
      else conv_result[bid]=0.0;
  }
  __syncthreads();
  
}

__global__
void Fdot8(float* fc1, float* w1, float* fc2, float *fc1_bias){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();
  sdata[tid]=fc1[tid]*w1[4096*tid+bid]+fc1[tid+1024]*w1[4096*(tid+1024)+bid]+fc1[tid+2048]*w1[4096*(tid+2048)+bid]+fc1[tid+3072]*w1[4096*(tid+3072)+bid];
  __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+fc1_bias[bid]>0) fc2[bid] = sdata[0]+fc1_bias[bid];
      else fc2[bid]=0.0;
  }
  __syncthreads();
  
}


void forward() {
   float tmp;
   poolmem tmppool;
   clock_t t1, t2;
   t1 = clock();

   //conv1 filtering

   for (int i = 0; i < 96; i++) {
        Fdot5<<<6561, 512>>>(conv1_input, conv1_filter, conv1_result, i, conv1_bias[i]);
       cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv1 pooling
   
   for (int k = 0; k < 96; k++) {
      for (int i = 0; i < 40; i += 1) {
         for (int j = 0; j < 40; j += 1) {
            tmp = conv1_result[83*96*(2 * i)+96*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<80)&&(2*i+a<80) && tmp <= conv1_result[83*96*(2 * i + a)+96*(2 * j + b)+k]) {
                     tmp = conv1_result[83*96*(2 * i + a)+96*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv1_maxpool[40*96*i+96*j+k]=tmppool;
            conv2_input[40*96*i+96*j+k] = tmp;
         }
      }
   }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv2 filtering
    
    for (int x = 0; x < 40; x++) {
         Fdot6<<<10240, 128>>>(conv2_input, conv2_filter, conv2_result, conv2_bias, x, 40, 40, 256, 5, 5, 96, 2);
         cudaDeviceSynchronize();
    }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   //conv2 pooling
   for (int k = 0; k < 256; k++) {
      for (int i = 0; i < 20; i += 1) {
         for (int j = 0; j < 20; j += 1) {
            tmp = conv2_result[42*256*(2 * i)+256*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<40)&&(2*i+a<40) && tmp <= conv2_result[42*256*(2 * i + a)+256*(2 * j + b)+k]) {
                     tmp = conv2_result[42*256*(2 * i + a)+256*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv3_input[20*256*i+256*j+k] = tmp;
            conv2_maxpool[20*256*i+256*j+k]=tmppool;
         }
      }
   }
   

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   //conv3 filtering
    for (int x = 0; x < 20; x++) {
         Fdot6<<<7680, 256>>>(conv3_input, conv3_filter, conv3_result, conv3_bias, x, 20, 20, 384, 3, 3, 256, 0);
         cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv4 filtering

    for (int x = 0; x < 20; x++) {
         Fdot6<<<7680, 512>>>(conv3_result, conv4_filter, conv4_result, conv4_bias, x, 20, 20, 384, 3, 3, 384, 0);
         cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv5 filtering
    for (int x = 0; x < 20; x++) {
         Fdot6<<<5120, 512>>>(conv4_result, conv5_filter, conv5_result, conv5_bias, x, 20, 20, 256, 3, 3, 384, 2);
         cudaDeviceSynchronize();
    }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //conv5 pooling
   for (int k = 0; k < 256; k++) {
      for (int i = 0; i < 10; i += 1) {
         for (int j = 0; j < 10; j += 1) {
            tmp = conv5_result[22*256*(2 * i)+256*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<20)&&(2*i+a<20) && tmp <= conv5_result[22*256*(2 * i + a)+256*(2 * j + b)+k]){
                     tmp = conv5_result[22*256*(2 * i + a)+256*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv5_maxpool[10*256*i+256*j+k]=tmppool;
         
            conv6_input[10*256*i+256*j+k] = tmp;
         }
      }
   }

    
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   

    //conv6 filtering
   
   Fdot7<<<4096, 256>>>(conv6_input, conv6_filter, fc1, conv6_bias);
     cudaDeviceSynchronize();
    

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //fc1
/*
   Fdot8<<<4096, 1024>>>(fc1, w1, fc2,fc1_bias);
     cudaDeviceSynchronize();
*/   

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //fc2
    for(int i=0; i<6; i++){
        sum=0;
        for(int j=0; j<4096; j++){
            sum+=fc1[j]*w2[6*j+i];
        }
        fc3[i]=exp(sum+fc2_bias[i]);
        //printf("%f\n",sum);
    }

   fc3[2]=0.0;

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    sum=0;

    for(int i=0; i<6; i++) sum+=fc3[i];

    //for(int i=0; i<6; i++) printf("%f ",fc3[i]/sum);
    
   //for(int i=0; i<6; i++) fprintf(fp,"%lf ",fc3[i]/sum);
   
   //printf("\n");
   
    
}

void backpropagation(int ans){
   float tmp;
   poolmem tmppool;
   clock_t t1, t2;
   t1 = clock();
   for(int i=0; i<6; i++) fc3_g[i]=(fc3[i]/sum)+(i==ans? -1:0);
   

   //second FC

   for(int i=0; i<4096; i++){
      tmp=0;
      for(int j=0; j<6; j++){
         tmp+=w2[6*i+j]*fc3_g[j];
      }
      fc1_g[i]=tmp;
   }


   for(int i=0; i<4096; i++){
      for(int j=0; j<6; j++){
         w2[6*i+j] -= LR*fc1[i]*fc3_g[j];
      }
   }

   for(int i=0; i<6; i++) fc2_bias[i] -= LR*fc3_g[i];


    t2 = clock();
   //printf("1: %lf\n", (float(t2 - t1)) / 1000);


   
   for(int i=0;i<4096; i++) if(fc1[i]<=0) fc1_g[i]=0;
   

   
    t2 = clock();
   //printf("2: %lf\n", (float(t2 - t1)) / 1000);

   
   //conv6

   Fdot3<<<25600,1024>>>(fc1_g, conv6_input_g, conv6_filter, LR);
   cudaDeviceSynchronize();



   Fdot2<<<25600,1024>>>(fc1_g, conv6_input, conv6_filter, LR);
   cudaDeviceSynchronize();

   for(int i=0; i<4096; i++) conv6_bias[i] -= LR*fc1_g[i];


    t2 = clock();
   //printf("3: %lf\n", (float(t2 - t1)) / 1000);


   //max_pooling과 relu 벗기기
   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<256; k++){
            conv5_result_g[20*256*i+256*j+k]=0;
         }
      }
   }

   for(int i=0; i<10; i++){
      for(int j=0; j<10; j++){
         for(int k=0; k<256; k++){
            tmppool=conv5_maxpool[10*256*i+256*j+k];
            if(conv5_result[22*256*tmppool.p+256*tmppool.q+tmppool.r]>0) conv5_result_g[20*256*tmppool.p+256*tmppool.q+tmppool.r] += conv6_input_g[10*256*i+256*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   ///conv5
   for(int i=0; i<20; i++){
       Fdot<<<7680,256>>> (conv5_filter, conv5_result_g, conv5_input_g, i ,20,20,384, 3, 3, 256);
       cudaDeviceSynchronize();
    }


      
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<384; k++){
            conv4_result_g[20*384*i+384*j+k] = conv4_result[20*384*i+384*j+k]>0?conv5_input_g[20*384*i+384*j+k]:0;
         }
      }
   }
   
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   
   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<12288,512>>>(conv5_result_g, conv4_result, conv5_filter, a, b, t, 20, 20, 384, 3, 3, 256, LR);
            cudaDeviceSynchronize();
         }   
      }
   }


    t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);

   ///conv4
   for(int i=0; i<20; i++){
       Fdot<<<7680,384>>> (conv4_filter, conv4_result_g, conv4_input_g, i ,20,20,384, 3, 3, 384);
       cudaDeviceSynchronize();
    }


    t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<384; k++){
            conv3_result_g[20*384*i+384*j+k] = conv3_result[20*384*i+384*j+k]>0?conv4_input_g[20*384*i+384*j+k]:0;
         }
      }
   }


    t2 = clock();
 t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);


   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<18432,512>>>(conv4_result_g, conv3_result, conv4_filter, a, b, t, 20, 20, 384, 3, 3, 384, LR);
            cudaDeviceSynchronize();
         }   
      }
   }


    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   ///conv3
   for(int i=0; i<20; i++){
       Fdot<<<5120 ,384>>> (conv3_filter, conv3_result_g, conv3_input_g, i ,20,20,256, 3, 3, 384);
       cudaDeviceSynchronize();
    }

    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<12288,512>>>(conv3_result_g, conv3_input, conv3_filter, a, b, t, 20, 20, 256, 3, 3, 384, LR);
            cudaDeviceSynchronize();
         }   
      }
   }

    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<40; i++){
      for(int j=0; j<40; j++){
         for(int k=0; k<256; k++){
            conv2_result_g[40*256*i+256*j+k]=0;
         }
      }
   }
   
    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);
   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<256; k++){
            tmppool=conv2_maxpool[20*256*i+256*j+k];
            if(conv2_result[42*256*tmppool.p+256*tmppool.q+tmppool.r]>0) conv2_result_g[40*256*tmppool.p+256*tmppool.q+tmppool.r] += conv3_input_g[20*256*i+256*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("7: %lf\n", (float(t2 - t1)) / 1000);

   ///conv2
   for(int i=0; i<40; i++){
       Fdot<<<3840 ,256>>> (conv2_filter, conv2_result_g, conv2_input_g, i ,40,40,96, 5, 5, 256);
       cudaDeviceSynchronize();
    }


       t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);

   for(int a=-2; a<=2; a++){
      for(int b=-2; b<=2; b++){
           Fdot1<<<24576,1024>>>(conv2_result_g, conv2_input, conv2_filter, a, b, 0, 40, 40, 96, 5, 5, 256, LR);
         cudaDeviceSynchronize();   
      }
   }

    t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);


   for(int i=0; i<81; i++){
      for(int j=0; j<81; j++){
         for(int k=0; k<96; k++){
            conv1_result_g[81*96*i+96*j+k]=0;
         }
      }
   }

   for(int i=0; i<40; i++){
      for(int j=0; j<40; j++){
         for(int k=0; k<96; k++){
            tmppool=conv1_maxpool[40*96*i+96*j+k];
            if(conv1_result[83*96*tmppool.p+96*tmppool.q+tmppool.r]>0) conv1_result_g[81*96*tmppool.p+96*tmppool.q+tmppool.r] += conv2_input_g[40*96*i+96*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);

   ///conv1
   
   
     Fdot4<<<34848,1024>>>(conv1_result_g, conv1_input, conv1_filter, LR);
     cudaDeviceSynchronize();

   

   for(int c=0; c<256; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv5_result_g[20*256*i+256*j+c];
         }
      }
      conv5_bias[c] -= tmp*LR;
   }

   for(int c=0; c<384; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv4_result_g[20*384*i+384*j+c];
         }
      }
      conv4_bias[c] -= tmp*LR;
   }


   for(int c=0; c<384; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv3_result_g[20*384*i+384*j+c];
         }
      }
      conv3_bias[c] -= tmp*LR;
   } 

   for(int c=0; c<256; c++){
      tmp=0.0;
      for(int i=0; i<40; i++){
         for(int j=0; j<40; j++){
            tmp += conv2_result_g[40*256*i+256*j+c];
         }
      }
      conv2_bias[c] -= tmp*LR;
   }  
   for(int c=0; c<96; c++){
      tmp=0.0;
      for(int i=0; i<80; i++){
         for(int j=0; j<80; j++){
            tmp += conv1_result_g[81*96*i+96*j+c];
         }
      }
      conv1_bias[c] -= tmp*LR;
   }
   

    t2 = clock();
   //printf("9: %lf\n", (float(t2 - t1)) / 1000);
   
}


int main() {
   

   cudaMallocManaged(&conv1_input, sizeof(float)*336*336*3);
   cudaMallocManaged(&conv1_filter, sizeof(float)*11*11*3*96);
   cudaMallocManaged(&conv1_bias, sizeof(float)*96);
   cudaMallocManaged(&conv1_result, sizeof(float)*83*83*96);

   cudaMallocManaged(&conv2_input, sizeof(float)*40*40*96);
   cudaMallocManaged(&conv2_filter, sizeof(float)*5*5*96*256);
   cudaMallocManaged(&conv2_bias, sizeof(float)*256);
   cudaMallocManaged(&conv2_result, sizeof(float)*42*42*256); //pooling에서 오류 대비

   cudaMallocManaged(&conv3_input, sizeof(float)*20*20*256);
   cudaMallocManaged(&conv3_filter, sizeof(float)*3*3*256*384);
   cudaMallocManaged(&conv3_bias, sizeof(float)*384);
   cudaMallocManaged(&conv3_result, sizeof(float)*20*20*384);
   //no pooling
   //float conv4_input, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv4_filter, sizeof(float)*3*3*384*384);
   cudaMallocManaged(&conv4_bias, sizeof(float)*384);
   cudaMallocManaged(&conv4_result, sizeof(float)*20*20*384);
   //no pooling
   //float conv5_input, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv5_filter, sizeof(float)*3*3*384*256);
   cudaMallocManaged(&conv5_bias, sizeof(float)*256);
   cudaMallocManaged(&conv5_result, sizeof(float)*22*22*256);

   cudaMallocManaged(&conv6_input, sizeof(float)*10*10*256);
   cudaMallocManaged(&conv6_filter, sizeof(float)*10*10*256*4096);
   cudaMallocManaged(&conv6_bias, sizeof(float)*4096);

   cudaMallocManaged(&fc1, sizeof(float)*4096);
   cudaMallocManaged(&w1, sizeof(float)*4096*4096);
   cudaMallocManaged(&fc1_bias, sizeof(float)*4096);

   cudaMallocManaged(&fc2, sizeof(float)*4096);
   cudaMallocManaged(&w2, sizeof(float)*4096*6);
   cudaMallocManaged(&fc2_bias, sizeof(float)*6);

   cudaMallocManaged(&fc3, sizeof(float)*6);

   cudaMallocManaged(&fc3_g, sizeof(float)*6);
   cudaMallocManaged(&fc2_g, sizeof(float)*4096);
   cudaMallocManaged(&fc1_g, sizeof(float)*4096);
   cudaMallocManaged(&conv6_input_g, sizeof(float)*10*10*256);

   cudaMallocManaged(&conv5_result_g, sizeof(float)*20*20*256);
   cudaMallocManaged(&conv5_input_g, sizeof(float)*20*20*384);
   //float conv5_input_g[20*20*384];

   cudaMallocManaged(&conv4_result_g, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv4_input_g, sizeof(float)*20*20*384);
   //float conv4_input_g[20*20*384];

   cudaMallocManaged(&conv3_result_g, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv3_input_g, sizeof(float)*20*20*256);

   cudaMallocManaged(&conv2_result_g, sizeof(float)*40*40*256);
   cudaMallocManaged(&conv2_input_g, sizeof(float)*40*40*96);

   cudaMallocManaged(&conv1_result_g, sizeof(float)*81*81*96);


   int num_testset=0;
   int randnum;
   pair <int, int> testset[2*CAT1_test+2*CAT2_test+2*CAT3_test+2*CAT4_test+2*CAT5_test+CAT6_test+10];
   clock_t t1,t2,start;
   t1=clock();
   start=clock();
   //fp=fopen("result.txt","w");
   srand(time(NULL));
   //printf("init\n");
   initdata(1);
   t2=clock();
   //printf("%lf\n",((float)(t2-t1))/1000);
   t1=clock();

   for(int i=1; i<=2*CAT1_test; i++){
      testset[num_testset++]=make_pair(1, i);
   }
   for(int i=1; i<=2*CAT2_test; i++){
      testset[num_testset++]=make_pair(2, i);
   }
   for(int i=1; i<=2*CAT3_test; i++){
      testset[num_testset++]=make_pair(3, i);
   }
   for(int i=1; i<=2*CAT4_test; i++){
      testset[num_testset++]=make_pair(4, i);
   }
   for(int i=1; i<=2*CAT5_test; i++){
      testset[num_testset++]=make_pair(5, i);
   }
   for(int i=1; i<=CAT6_test; i++){
      testset[num_testset++]=make_pair(6, i);
   }

   //데이터셋 섞기
   t1=clock();

   //데이터셋 학습
   for(int i=0; i<epoch; i++){
      if(i!=0 && i%(epoch/10)==0) LR-=0.000001;
      randnum=i%6+1;
      if(randnum==3) {
         continue;
      }
      //printf("\n%d try",i);
      //t1=clock();
      //printf("\ngetimage\n");
      getimage(randnum,rand()%datasize(randnum)+1);
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
   
      //t1=clock();
      //printf("\nforward\n");
      forward();
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
      
      //printf("backpropagation\n");
      //t1=clock();
      backpropagation(randnum-1);
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
      
   }
      
   t2=clock();
   printf("%f\n\n",((float)(t2-t1))/1000000.0);

   for(int i=0; i<num_testset; i++){
      printf("\n[CAT%d %d]",testset[i].first,testset[i].second);
     gettestimage(testset[i].first,testset[i].second);
      forward();
      if(testset[i].second>20) checkanswer1(testset[i].first-1);
     else checkanswer(testset[i].first-1);
   }
   printf("\nRESULT\n\n");
   printf("TOT1: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4]+testresult[5])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test+CAT6_test));
   printf("TOT2: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)testresult[0])/CAT1_test);
   printf("CAT2: %f\n",((float)testresult[1])/CAT2_test);
   //printf("CAT3: %f\n",((float)testresult[2])/CAT3_test);
   printf("CAT4: %f\n",((float)testresult[3])/CAT4_test);
   printf("CAT5: %f\n",((float)testresult[4])/CAT5_test);
   printf("CAT6: %f\n\n",((float)testresult[5])/CAT6_test);

   if(catresultcat[0]+catresultcat[1]+catresultcat[3]+catresultcat[4]!=0) printf("TOT3: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4])/(catresultcat[0]+catresultcat[1]+catresultcat[3]+catresultcat[4]));
   if(catresultcat[0]!=0) printf("CAT1: %f\n",((float)testresult[0])/catresultcat[0]);
   if(catresultcat[1]!=0) printf("CAT2: %f\n",((float)testresult[1])/catresultcat[1]);
   //if(catresultcat[2]!=0) printf("CAT3: %f\n",((float)testresult[2])/catresultcat[2]);
   if(catresultcat[3]!=0) printf("CAT4: %f\n",((float)testresult[3])/catresultcat[3]);
   if(catresultcat[4]!=0) printf("CAT5: %f\n\n",((float)testresult[4])/catresultcat[4]);

   printf("TOT4: %f\n\n", ((float)onlycat[0]+onlycat[1]+onlycat[3]+onlycat[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)onlycat[0])/CAT1_test);
   printf("CAT2: %f\n",((float)onlycat[1])/CAT2_test);
   //printf("CAT3: %f\n",((float)onlycat[2])/CAT3_test);
   printf("CAT4: %f\n",((float)onlycat[3])/CAT4_test);
   printf("CAT5: %f\n",((float)onlycat[4])/CAT5_test);

      printf("\n\n\nRESULT2\n\n");
   printf("TOT2: %f\n\n", ((float)testresult1[0]+testresult1[1]+testresult1[3]+testresult1[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)testresult1[0])/CAT1_test);
   printf("CAT2: %f\n",((float)testresult1[1])/CAT2_test);
   printf("CAT4: %f\n",((float)testresult1[3])/CAT4_test);
   printf("CAT5: %f\n\n",((float)testresult1[4])/CAT5_test);

   if(catresultcat1[0]+catresultcat1[1]+catresultcat1[3]+catresultcat1[4]!=0) printf("TOT3: %f\n\n", ((float)testresult1[0]+testresult1[1]+testresult1[3]+testresult1[4])/(catresultcat1[0]+catresultcat1[1]+catresultcat1[3]+catresultcat1[4]));
   if(catresultcat1[0]!=0) printf("CAT1: %f\n",((float)testresult1[0])/catresultcat1[0]);
   if(catresultcat1[1]!=0) printf("CAT2: %f\n",((float)testresult1[1])/catresultcat1[1]);
   if(catresultcat1[3]!=0) printf("CAT4: %f\n",((float)testresult1[3])/catresultcat1[3]);
   if(catresultcat1[4]!=0) printf("CAT5: %f\n\n",((float)testresult1[4])/catresultcat1[4]);

   printf("TOT4: %f\n\n", ((float)onlycat1[0]+onlycat1[1]+onlycat1[3]+onlycat1[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)onlycat1[0])/CAT1_test);
   printf("CAT2: %f\n",((float)onlycat1[1])/CAT2_test);
   printf("CAT4: %f\n",((float)onlycat1[3])/CAT4_test);
   printf("CAT5: %f\n",((float)onlycat1[4])/CAT5_test);





      //printf("%f\n",((float)(t2-start))/1000000);
}

4031.172352


[CAT1 1]
0.205281 0.152839 0.000000 0.194348 0.249684 0.197849 
answer: 0
result: 4

[CAT1 2]
0.624268 0.034375 0.000000 0.025095 0.171085 0.145178 
answer: 0
result: 0

[CAT1 3]
0.369048 0.086385 0.000000 0.045191 0.251374 0.248001 
answer: 0
result: 0

[CAT1 4]
0.414703 0.059193 0.000000 0.028528 0.301406 0.196171 
answer: 0
result: 0

[CAT1 5]
0.409146 0.066454 0.000000 0.027709 0.315479 0.181212 
answer: 0
result: 0

[CAT1 6]
0.430860 0.095957 0.000000 0.035783 0.254321 0.183078 
answer: 0
result: 0

[CAT1 7]
0.115766 0.176137 0.000000 0.098045 0.178119 0.431933 
answer: 0
result: 5

[CAT1 8]
0.368532 0.069618 0.000000 0.190885 0.312002 0.058963 
answer: 0
result: 0

[CAT1 9]
0.013438 0.453136 0.000000 0.358884 0.084445 0.090096 
answer: 0
result: 1

[CAT1 10]
0.177726 0.219646 0.000000 0.066041 0.298749 0.237837 
answer: 0
result: 4

[CAT1 11]
0.068849 0.228824 0.000000 0.486594 0.098511 0.117221 
answer: 0
result: 3

[CAT1 12]
0.256297 0.204110 0.000000 0.219125 0.2

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
%%cu
#include <stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
#include<algorithm>
using namespace std;

#define HEADER 54
#define WIDTH 336
#define HEIGHT 336
#define CAT1 2192
#define CAT2 1800
#define CAT3 0
#define CAT4 1448
#define CAT5 1688
#define CAT6 1136

#define CAT1_test 20
#define CAT2_test 20
#define CAT3_test 0
#define CAT4_test 20
#define CAT5_test 20
#define CAT6_test 20


using namespace std;

typedef struct poolmem{
   int p,q,r;
}poolmem;

float* conv1_input;
float* conv1_filter;
float* conv1_bias;
float* conv1_result;

float* conv2_input;
float* conv2_filter;
float* conv2_bias;
float* conv2_result; //pooling에서 오류 대비

float* conv3_input;
float* conv3_filter;
float* conv3_bias;
float* conv3_result;
//no pooling
//float conv4_input[20*20*384];
float* conv4_filter;
float* conv4_bias;
float* conv4_result;
//no pooling
//float conv5_input[20*20*384];
float* conv5_filter;
float* conv5_bias;
float* conv5_result;

poolmem conv5_maxpool[10*10*256];
poolmem conv2_maxpool[20*20*256];
poolmem conv1_maxpool[40*40*96];
int testresult[6];
int catresultcat[6];
int onlycat[6];

float* conv6_input;
float* conv6_filter;
float* conv6_bias;

float* fc1;
float* w1;
float* fc1_bias;

float* fc2;
float* w2;
float* fc2_bias;

float* fc3;

float* fc3_g;
float* fc2_g;
float* fc1_g;
float* conv6_input_g;

float* conv5_result_g;
float* conv5_input_g;

float* conv4_result_g;
float* conv4_input_g;

float* conv3_result_g;
float* conv3_input_g;

float* conv2_result_g;
float* conv2_input_g;

float* conv1_result_g;
//float conv1_input_g[40][40][96];





FILE * fp;



float LR=0.000015;
int epoch=12000;
int result;

int datasize(int t){
   if(t==1) return CAT1;
   if(t==2) return CAT2;
   if(t==3) return CAT3;
   if(t==4) return CAT4;
   if(t==5) return CAT5;
   if(t==6) return CAT6;
   return 0;
}

void getimage(int cat, int num){
    char filename[100];
    if(cat<=2) sprintf(filename,"/content/gdrive/Shared drives/COSE362/dataset/e/CAT%d/cat%d_(%d).bmp",cat,cat,num);
    else sprintf(filename,"/content/gdrive/Shared drives/COSE362/dataset/e/CAT%d/CAT%d (%d).bmp",cat,cat,num);
    FILE *fp1 = fopen(filename,"rb");
    unsigned char tmp;
    for(int i=0; i<HEADER; i++) tmp=getc(fp1);
    for(int i=0; i<HEIGHT; i++)
        for(int j=0; j<WIDTH; j++)
        {
            conv1_input[336*3*i+3*j+2]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+1]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+0]=((float)getc(fp1))/256;
        }
   fclose(fp1);
}

void gettestimage(int cat, int num){
    char filename[100];
    if(cat<=2) sprintf(filename,"/content/gdrive/Shared drives/COSE362/testset/CAT%d/cat%d (%d).bmp",cat,cat,num);
   else sprintf(filename,"/content/gdrive/Shared drives/COSE362/testset/CAT%d/CAT%d (%d).bmp",cat,cat,num);
    FILE *fp1 = fopen(filename,"rb");
    unsigned char tmp;
    for(int i=0; i<HEADER; i++) tmp=getc(fp1);
    for(int i=0; i<HEIGHT; i++)
        for(int j=0; j<WIDTH; j++)
        {
            conv1_input[336*3*i+3*j+2]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+1]=((float)getc(fp1))/256;
            conv1_input[336*3*i+3*j+0]=((float)getc(fp1))/256;
        }
   fclose(fp1);
}

float sum;

int testresult1[6];
int catresultcat1[6];
int onlycat1[6];

void checkanswer1(int ans){
   int tmp=0;
   printf("\n");
   for(int i=0; i<6; i++){
      printf("%f ",fc3[i]/sum);
      if(fc3[tmp]<=fc3[i]) {
         tmp=i;
      }
   }
   printf("\nanswer: %d\nresult: %d\n",ans,tmp);
   if(tmp==ans){
      testresult1[ans]++;
   }
   if(ans<5 && tmp<5){
      catresultcat1[ans]++;
   }
   if(ans<5){
      int tmp=0;
      for(int i=0; i<5; i++){
         if(fc3[tmp]<=fc3[i]) {
            tmp=i;
         }
      }
      if(tmp==ans) onlycat1[ans]++;
   }
}
void checkanswer(int ans){
   int tmp=0;
   printf("\n");
   for(int i=0; i<6; i++){
      printf("%f ",fc3[i]/sum);
      if(fc3[tmp]<=fc3[i]) {
         tmp=i;
      }
   }
   printf("\nanswer: %d\nresult: %d\n",ans,tmp);
   if(tmp==ans){
      testresult[ans]++;
   }
   if(ans<5 && tmp<5){
      catresultcat[ans]++;
   }
   if(ans<5){
      int tmp=0;
      for(int i=0; i<5; i++){
         if(fc3[tmp]<=fc3[i]) {
            tmp=i;
         }
      }
      if(tmp==ans) onlycat[ans]++;
   }
}

float randrange(float t){
    return (float)(2.0*t*((((double)rand())/2147483647.0)-0.5));
}

void initdata(int mode){
   for(int i=0; i<96; i++) conv1_bias[i]=0.0;
   for(int i=0; i<256; i++) conv2_bias[i]=0.0;
   for(int i=0; i<384; i++) conv3_bias[i]=0.0;
   for(int i=0; i<384; i++) conv4_bias[i]=0.0;
   for(int i=0; i<256; i++) conv5_bias[i]=0.0;
   for(int i=0; i<4096; i++) conv6_bias[i]=0.0;
   for(int i=0; i<4096; i++) fc1_bias[i]=0.0;
   for(int i=0; i<6; i++) fc2_bias[i]=0.0;
    
    if(mode==0){
        FILE* fp = fopen("/content/gdrive/Shared drives/COSE362/testset/init.txt","w");
        for(int i=0; i<11; i++){
            for(int j=0; j<11; j++){
                for(int k=0; k<3; k++){
                    for(int t=0; t<96; t++) fprintf(fp,"%f\n",conv1_filter[11*3*96*i+3*96*j+96*k+t]=randrange(sqrt(((float)6.0)/363)));
                }
            }
        }
        
        for(int i=0; i<5; i++){
            for(int j=0; j<5; j++){
                for(int k=0; k<96; k++){
                    for(int t=0; t<256; t++) fprintf(fp,"%f\n",conv2_filter[5*96*256*i+96*256*j+256*k+t]=randrange(sqrt(((float)6.0)/2400)));
                }
            }
        }
        
        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<384; t++) fprintf(fp,"%f\n",conv3_filter[3*256*384*i+256*384*j+384*k+t]=randrange(sqrt(((float)6.0)/2304)));
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<384; t++) fprintf(fp,"%f\n",conv4_filter[3*384*384*i+384*384*j+384*k+t]=randrange(sqrt(((float)6.0)/3456)));
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<256; t++) fprintf(fp,"%f\n",conv5_filter[3*384*256*i+384*256*j+256*k+t]=randrange(sqrt(((float)6.0)/3456)));
                }
            }
        }

        for(int i=0; i<10; i++){
            for(int j=0; j<10; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<4096; t++) fprintf(fp,"%f\n",conv6_filter[10*256*4096*i+256*4096*j+4096*k+t]=randrange(sqrt(((float)6.0)/25600)));
                }
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<4096; j++){
                fprintf(fp,"%f\n",w1[4096*i+j]=randrange(sqrt(((float)6.0)/4096)));
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<6; j++){
                fprintf(fp,"%f\n",w2[6*i+j]=randrange(sqrt(((float)6.0)/4096)));
            }
        }
        fclose(fp);
    }
    else if(mode==1){
        FILE* fp = fopen("/content/gdrive/Shared drives/COSE362/testset/init.txt","r");
        for(int i=0; i<11; i++){
            for(int j=0; j<11; j++){
                for(int k=0; k<3; k++){
                    for(int t=0; t<96; t++) fscanf(fp,"%f",&conv1_filter[11*3*96*i+3*96*j+96*k+t]);
                }
            }
        }
        
        for(int i=0; i<5; i++){
            for(int j=0; j<5; j++){
                for(int k=0; k<96; k++){
                    for(int t=0; t<256; t++) fscanf(fp,"%f",&conv2_filter[5*96*256*i+96*256*j+256*k+t]);
                }
            }
        }
        
        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<384; t++) fscanf(fp,"%f",&conv3_filter[3*256*384*i+256*384*j+384*k+t]);
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<384; t++) fscanf(fp,"%f",&conv4_filter[3*384*384*i+384*384*j+384*k+t]);
                }
            }
        }

        for(int i=0; i<3; i++){
            for(int j=0; j<3; j++){
                for(int k=0; k<384; k++){
                    for(int t=0; t<256; t++) fscanf(fp,"%f",&conv5_filter[3*384*256*i+384*256*j+256*k+t]);
                }
            }
        }

        for(int i=0; i<10; i++){
            for(int j=0; j<10; j++){
                for(int k=0; k<256; k++){
                    for(int t=0; t<4096; t++) fscanf(fp,"%f",&conv6_filter[10*256*4096*i+256*4096*j+4096*k+t]);
                }
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<4096; j++){
                fscanf(fp,"%f",&w1[4096*i+j]);
            }
        }

        for(int i=0; i<4096; i++){
            for(int j=0; j<6; j++){
                fscanf(fp,"%f",&w2[6*i+j]);
            }
        }
        fclose(fp);
    }
}


__device__ void warpReduce(volatile float* sdata, int tid) {
sdata[tid] += sdata[tid + 32];
sdata[tid] += sdata[tid + 16];
sdata[tid] += sdata[tid + 8];
sdata[tid] += sdata[tid + 4];
sdata[tid] += sdata[tid + 2];
sdata[tid] += sdata[tid + 1];
}

__global__
void Fdot(float* conv_filter, float* conv_result_g, float* conv_input_g, int i, int I, int J, int K, int A, int B, int C){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  
   unsigned int j = (blockIdx.x/K)%J;
   unsigned int k = blockIdx.x%K;
  sdata[tid]=0.0;
  float init=0.0;
   for(int a1=-(A/2); a1<=(A/2); a1++){
      for(int b1=-(B/2); b1<=(B/2); b1++){
           __syncthreads();
          if((i+a1)>=0 && (i+a1)<I && (j+b1)>=0 && (j+b1)<J) init += conv_filter[B*K*C*((A/2)-a1)+K*C*((B/2)-b1)+C*k+tid]*conv_result_g[J*C*(i+a1)+C*(j+b1)+tid];
         __syncthreads();
      }
   }
  
    __syncthreads();
    sdata[tid] = init;
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
    __syncthreads();
  
  __syncthreads();
  if(tid==0) conv_input_g[i*J*K+blockIdx.x] = sdata[tid];
  __syncthreads();
}

__global__
void Fdot1(float* conv_result_g, float* conv_input, float* conv_filter, int a, int b, int t, int I, int J, int K, int A, int B, int C, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i = tid/J;
  unsigned int j = tid%J;
  unsigned int c = (bid)%C;
  unsigned int k = (bid)/C;
  
    __syncthreads();
    sdata[tid] = 0.0;
    __syncthreads();
    if(tid<800 && (i+a)>=0 && (i+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] += conv_result_g[J*C*i+C*j+c]*conv_input[J*K*(i+a)+K*(j+b)+k];
    __syncthreads();
    if(tid<800 && (i+20+a)>=0 && (i+20+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] += conv_result_g[J*C*(i+20)+C*j+c]*conv_input[J*K*(i+a+20)+K*(j+b)+k];
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>32; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
   __syncthreads();
 if (tid < 32) warpReduce(sdata, tid);
  __syncthreads();
  if(tid==0) conv_filter[B*K*C*((A/2)+a)+K*C*((B/2)+b)+C*k+c] -= sdata[0]*LR;
  __syncthreads();
}

__global__
void Fdot11(float* conv_result_g, float* conv_input, float* conv_filter, int a, int b, int t, int I, int J, int K, int A, int B, int C, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i = tid/J;
  unsigned int j = tid%J;
  unsigned int c = bid%C;
  unsigned int k = bid/C + t*(K/8);
  
    __syncthreads();
    if(tid<400 && (i+a)>=0 && (i+a)<I && (j+b)>=0 && (j+b)<J) sdata[tid] = conv_result_g[J*C*i+C*j+c]*conv_input[J*K*(i+a)+K*(j+b)+k];
    else sdata[tid] = 0.0;
    __syncthreads();
  
  for (unsigned int s=blockDim.x/2; s>32; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
    __syncthreads();
  if (tid < 32) warpReduce(sdata, tid);
  __syncthreads();
  if(tid==0) conv_filter[B*K*C*((A/2)+a)+K*C*((B/2)+b)+C*k+c] -= sdata[0]*LR;
  __syncthreads();
}

__global__
void Fdot2(float* fc_g, float* conv_input, float* conv_filter, float LR){
  __shared__ float tmp;
   unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();   
  if(tid==0) tmp=conv_input[bid];
  __syncthreads();   
  for(int t=0; t<4096; t+=1024){
      __syncthreads();
        conv_filter[4096*bid+tid+t] -= LR*tmp*fc_g[tid+t];
       __syncthreads();
  }
}

__global__
void Fdot3(float* fc_g, float* conv_input_g, float* conv_filter, float LR){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();   
  sdata[tid] = conv_filter[4096*bid+tid]*fc_g[tid] + conv_filter[4096*bid+tid+1024]*fc_g[tid+1024] + conv_filter[4096*bid+tid+2048]*fc_g[tid+2048] + conv_filter[4096*bid+tid+3072]*fc_g[tid+3072];
  __syncthreads();   
  
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    }
    __syncthreads();
  }
  __syncthreads();
  if(tid==0) conv_input_g[bid] = sdata[0];
  __syncthreads();
  
}

__global__
void Fdot4(float* conv_result_g, float* conv_input, float* conv_filter, float LR){
  __shared__ float sdata[1024];
  float init=0.0;
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int a=(bid/(3*96*11))%11;
  unsigned int b=(bid/(3*96))%11;
  unsigned int k=(bid/96)%3;
  unsigned int c=bid%96;
  int i;
  int j;
  __syncthreads();   
  for(int t=0; t<7; t++){
      __syncthreads();   
      if(tid+1024*t<6561) {
      __syncthreads();   
      i=((tid+1024*t)/81)%81;
      __syncthreads();   
      j=(tid+1024*t)%81;
      __syncthreads();   
      init += conv_input[336*3*(4*i+a)+3*(4*j+b)+k]*conv_result_g[81*96*i+96*j+c];
      __syncthreads();   
    }
    __syncthreads();   
   }
  __syncthreads();
  sdata[tid] = init;   
  __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();
  if(tid==0) conv_filter[bid] -= sdata[0]*LR;
  __syncthreads();
  
}

__global__
void Fdot5(float* conv_input, float* conv_filter, float* conv_result, int i, float bias){
  __shared__ float sdata[1024];
  float init=0.0;
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int x=(bid/81)%81;
  unsigned int y=bid%81;
  int a,b,c;
  __syncthreads();   
  if(tid<363){
  __syncthreads();   
    c=tid%3;
  __syncthreads();   
    b=(tid/3)%11;
  __syncthreads();   
    a=(tid/33)%11;
  __syncthreads();   
    sdata[tid]=conv_input[336*3*(4*x + a)+3*(4*y + b)+c] * conv_filter[11*3*96*a+3*96*b+96*c+i];
  __syncthreads();   
  }
  else sdata[tid]=0;
  __syncthreads();   
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+bias>0) conv_result[83*96*x+96*y+i] = sdata[0]+bias;
      else conv_result[83*96*x+96*y+i]=0.0;
  }
  __syncthreads();
  
}


__global__
void Fdot6(float* conv_input, float* conv_filter, float* conv_result, float *conv_bias, int x, int X, int Y, int I, int A, int B, int C, int extra){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  unsigned int i=bid%I;
  unsigned int y=(bid/I)%Y;
  float init;
  
  if(tid<C){
    init=0.0;
    __syncthreads();   
    for(int a=-(A/2); a<=A/2; a++){
        __syncthreads();   
        for(int b=-(B/2); b<=B/2; b++){
            __syncthreads();   
            if((x+a)>=0 && (x+a)<X && (y+b)>=0 && (y+b)<Y) init += conv_input[Y*C*(x + a)+C*(y + b)+tid] * conv_filter[A*C*I*(a+(A/2))+C*I*(b+(B/2))+I*tid+i];
             __syncthreads();   
        }
      __syncthreads();   
      }
    sdata[tid]=init;
    __syncthreads();   
  }
  else sdata[tid]=0;
  __syncthreads();   
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+conv_bias[i]>0) conv_result[(X+extra)*I*x+I*y+i] = sdata[0]+conv_bias[i];
      else conv_result[(X+extra)*I*x+I*y+i]=0.0;
  }
  __syncthreads();
  
}

__global__
void Fdot7(float* conv_input, float* conv_filter, float* conv_result, float *conv_bias){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  float init=0.0;
  __syncthreads();
      for(int a=0; a<10; a++){
      __syncthreads();
      for(int b=0; b<10; b++){
          __syncthreads();
          init += conv_input[10*256*a+256*b+tid] * conv_filter[10*256*4096*a+256*4096*b+4096*tid+bid];
         __syncthreads();
      }
      __syncthreads();
  }
    __syncthreads();
  sdata[tid]=init;
    __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+conv_bias[bid]>0) conv_result[bid] = sdata[0]+conv_bias[bid];
      else conv_result[bid]=0.0;
  }
  __syncthreads();
  
}

__global__
void Fdot8(float* fc1, float* w1, float* fc2, float *fc1_bias){
  __shared__ float sdata[1024];
  unsigned int tid = threadIdx.x;
  unsigned int bid = blockIdx.x;
  __syncthreads();
  sdata[tid]=fc1[tid]*w1[4096*tid+bid]+fc1[tid+1024]*w1[4096*(tid+1024)+bid]+fc1[tid+2048]*w1[4096*(tid+2048)+bid]+fc1[tid+3072]*w1[4096*(tid+3072)+bid];
  __syncthreads();
  for (unsigned int s=blockDim.x/2; s>0; s>>=1) {
    if (tid < s) {
      sdata[tid] += sdata[tid + s];
    __syncthreads();
    }
    __syncthreads();
  }
  __syncthreads();

  if(tid==0) {
      if(sdata[0]+fc1_bias[bid]>0) fc2[bid] = sdata[0]+fc1_bias[bid];
      else fc2[bid]=0.0;
  }
  __syncthreads();
  
}


void forward() {
   float tmp;
   poolmem tmppool;
   clock_t t1, t2;
   t1 = clock();

   //conv1 filtering

   for (int i = 0; i < 96; i++) {
        Fdot5<<<6561, 512>>>(conv1_input, conv1_filter, conv1_result, i, conv1_bias[i]);
       cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv1 pooling
   
   for (int k = 0; k < 96; k++) {
      for (int i = 0; i < 40; i += 1) {
         for (int j = 0; j < 40; j += 1) {
            tmp = conv1_result[83*96*(2 * i)+96*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<80)&&(2*i+a<80) && tmp <= conv1_result[83*96*(2 * i + a)+96*(2 * j + b)+k]) {
                     tmp = conv1_result[83*96*(2 * i + a)+96*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv1_maxpool[40*96*i+96*j+k]=tmppool;
            conv2_input[40*96*i+96*j+k] = tmp;
         }
      }
   }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv2 filtering
    
    for (int x = 0; x < 40; x++) {
         Fdot6<<<10240, 128>>>(conv2_input, conv2_filter, conv2_result, conv2_bias, x, 40, 40, 256, 5, 5, 96, 2);
         cudaDeviceSynchronize();
    }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   //conv2 pooling
   for (int k = 0; k < 256; k++) {
      for (int i = 0; i < 20; i += 1) {
         for (int j = 0; j < 20; j += 1) {
            tmp = conv2_result[42*256*(2 * i)+256*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<40)&&(2*i+a<40) && tmp <= conv2_result[42*256*(2 * i + a)+256*(2 * j + b)+k]) {
                     tmp = conv2_result[42*256*(2 * i + a)+256*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv3_input[20*256*i+256*j+k] = tmp;
            conv2_maxpool[20*256*i+256*j+k]=tmppool;
         }
      }
   }
   

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   //conv3 filtering
    for (int x = 0; x < 20; x++) {
         Fdot6<<<7680, 256>>>(conv3_input, conv3_filter, conv3_result, conv3_bias, x, 20, 20, 384, 3, 3, 256, 0);
         cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv4 filtering

    for (int x = 0; x < 20; x++) {
         Fdot6<<<7680, 512>>>(conv3_result, conv4_filter, conv4_result, conv4_bias, x, 20, 20, 384, 3, 3, 384, 0);
         cudaDeviceSynchronize();
    }

   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

   //conv5 filtering
    for (int x = 0; x < 20; x++) {
         Fdot6<<<5120, 512>>>(conv4_result, conv5_filter, conv5_result, conv5_bias, x, 20, 20, 256, 3, 3, 384, 2);
         cudaDeviceSynchronize();
    }
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //conv5 pooling
   for (int k = 0; k < 256; k++) {
      for (int i = 0; i < 10; i += 1) {
         for (int j = 0; j < 10; j += 1) {
            tmp = conv5_result[22*256*(2 * i)+256*(2 * j)+k];
            for (int a = 0; a < 3; a++) {
               for (int b = 0; b < 3; b++) {
                  if ((2*j+b<20)&&(2*i+a<20) && tmp <= conv5_result[22*256*(2 * i + a)+256*(2 * j + b)+k]){
                     tmp = conv5_result[22*256*(2 * i + a)+256*(2 * j + b)+k];
                     tmppool.p=2*i+a;
                     tmppool.q=2*j+b;
                     tmppool.r=k;
                  }
               }
            }
            conv5_maxpool[10*256*i+256*j+k]=tmppool;
         
            conv6_input[10*256*i+256*j+k] = tmp;
         }
      }
   }

    
   t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);


   

    //conv6 filtering
   
   Fdot7<<<4096, 256>>>(conv6_input, conv6_filter, fc1, conv6_bias);
     cudaDeviceSynchronize();
    

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //fc1
/*
   Fdot8<<<4096, 1024>>>(fc1, w1, fc2,fc1_bias);
     cudaDeviceSynchronize();
*/   

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    //fc2
    for(int i=0; i<6; i++){
        sum=0;
        for(int j=0; j<4096; j++){
            sum+=fc1[j]*w2[6*j+i];
        }
        fc3[i]=exp(sum+fc2_bias[i]);
        //printf("%f\n",sum);
    }

   fc3[2]=0.0;

    t2 = clock();
   //printf("%lf\n", (float(t2 - t1)) / 1000);

    sum=0;

    for(int i=0; i<6; i++) sum+=fc3[i];

    //for(int i=0; i<6; i++) printf("%f ",fc3[i]/sum);
    
   //for(int i=0; i<6; i++) fprintf(fp,"%lf ",fc3[i]/sum);
   
   //printf("\n");
   
    
}

void backpropagation(int ans){
   float tmp;
   poolmem tmppool;
   clock_t t1, t2;
   t1 = clock();
   for(int i=0; i<6; i++) fc3_g[i]=(fc3[i]/sum)+(i==ans? -1:0);
   

   //second FC

   for(int i=0; i<4096; i++){
      tmp=0;
      for(int j=0; j<6; j++){
         tmp+=w2[6*i+j]*fc3_g[j];
      }
      fc1_g[i]=tmp;
   }


   for(int i=0; i<4096; i++){
      for(int j=0; j<6; j++){
         w2[6*i+j] -= LR*fc1[i]*fc3_g[j];
      }
   }

   for(int i=0; i<6; i++) fc2_bias[i] -= LR*fc3_g[i];


    t2 = clock();
   //printf("1: %lf\n", (float(t2 - t1)) / 1000);


   
   for(int i=0;i<4096; i++) if(fc1[i]<=0) fc1_g[i]=0;
   

   
    t2 = clock();
   //printf("2: %lf\n", (float(t2 - t1)) / 1000);

   
   //conv6

   Fdot3<<<25600,1024>>>(fc1_g, conv6_input_g, conv6_filter, LR);
   cudaDeviceSynchronize();



   Fdot2<<<25600,1024>>>(fc1_g, conv6_input, conv6_filter, LR);
   cudaDeviceSynchronize();

   for(int i=0; i<4096; i++) conv6_bias[i] -= LR*fc1_g[i];


    t2 = clock();
   //printf("3: %lf\n", (float(t2 - t1)) / 1000);


   //max_pooling과 relu 벗기기
   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<256; k++){
            conv5_result_g[20*256*i+256*j+k]=0;
         }
      }
   }

   for(int i=0; i<10; i++){
      for(int j=0; j<10; j++){
         for(int k=0; k<256; k++){
            tmppool=conv5_maxpool[10*256*i+256*j+k];
            if(conv5_result[22*256*tmppool.p+256*tmppool.q+tmppool.r]>0) conv5_result_g[20*256*tmppool.p+256*tmppool.q+tmppool.r] += conv6_input_g[10*256*i+256*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   ///conv5
   for(int i=0; i<20; i++){
       Fdot<<<7680,256>>> (conv5_filter, conv5_result_g, conv5_input_g, i ,20,20,384, 3, 3, 256);
       cudaDeviceSynchronize();
    }


      
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<384; k++){
            conv4_result_g[20*384*i+384*j+k] = conv4_result[20*384*i+384*j+k]>0?conv5_input_g[20*384*i+384*j+k]:0;
         }
      }
   }
   
    t2 = clock();
   //printf("4: %lf\n", (float(t2 - t1)) / 1000);

   
   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<12288,512>>>(conv5_result_g, conv4_result, conv5_filter, a, b, t, 20, 20, 384, 3, 3, 256, LR);
            cudaDeviceSynchronize();
         }   
      }
   }


    t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);

   ///conv4
   for(int i=0; i<20; i++){
       Fdot<<<7680,384>>> (conv4_filter, conv4_result_g, conv4_input_g, i ,20,20,384, 3, 3, 384);
       cudaDeviceSynchronize();
    }


    t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<384; k++){
            conv3_result_g[20*384*i+384*j+k] = conv3_result[20*384*i+384*j+k]>0?conv4_input_g[20*384*i+384*j+k]:0;
         }
      }
   }


    t2 = clock();
 t2 = clock();
   //printf("5: %lf\n", (float(t2 - t1)) / 1000);


   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<18432,512>>>(conv4_result_g, conv3_result, conv4_filter, a, b, t, 20, 20, 384, 3, 3, 384, LR);
            cudaDeviceSynchronize();
         }   
      }
   }


    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   ///conv3
   for(int i=0; i<20; i++){
       Fdot<<<5120 ,384>>> (conv3_filter, conv3_result_g, conv3_input_g, i ,20,20,256, 3, 3, 384);
       cudaDeviceSynchronize();
    }

    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   for(int a=-1; a<=1; a++){
      for(int b=-1; b<=1; b++){
           for(int t=0; t<8; t++){
            Fdot11<<<12288,512>>>(conv3_result_g, conv3_input, conv3_filter, a, b, t, 20, 20, 256, 3, 3, 384, LR);
            cudaDeviceSynchronize();
         }   
      }
   }

    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);

   for(int i=0; i<40; i++){
      for(int j=0; j<40; j++){
         for(int k=0; k<256; k++){
            conv2_result_g[40*256*i+256*j+k]=0;
         }
      }
   }
   
    t2 = clock();
   //printf("6: %lf\n", (float(t2 - t1)) / 1000);
   for(int i=0; i<20; i++){
      for(int j=0; j<20; j++){
         for(int k=0; k<256; k++){
            tmppool=conv2_maxpool[20*256*i+256*j+k];
            if(conv2_result[42*256*tmppool.p+256*tmppool.q+tmppool.r]>0) conv2_result_g[40*256*tmppool.p+256*tmppool.q+tmppool.r] += conv3_input_g[20*256*i+256*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("7: %lf\n", (float(t2 - t1)) / 1000);

   ///conv2
   for(int i=0; i<40; i++){
       Fdot<<<3840 ,256>>> (conv2_filter, conv2_result_g, conv2_input_g, i ,40,40,96, 5, 5, 256);
       cudaDeviceSynchronize();
    }


       t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);

   for(int a=-2; a<=2; a++){
      for(int b=-2; b<=2; b++){
           Fdot1<<<24576,1024>>>(conv2_result_g, conv2_input, conv2_filter, a, b, 0, 40, 40, 96, 5, 5, 256, LR);
         cudaDeviceSynchronize();   
      }
   }

    t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);


   for(int i=0; i<81; i++){
      for(int j=0; j<81; j++){
         for(int k=0; k<96; k++){
            conv1_result_g[81*96*i+96*j+k]=0;
         }
      }
   }

   for(int i=0; i<40; i++){
      for(int j=0; j<40; j++){
         for(int k=0; k<96; k++){
            tmppool=conv1_maxpool[40*96*i+96*j+k];
            if(conv1_result[83*96*tmppool.p+96*tmppool.q+tmppool.r]>0) conv1_result_g[81*96*tmppool.p+96*tmppool.q+tmppool.r] += conv2_input_g[40*96*i+96*j+k];
         }
      }
   }
   
    t2 = clock();
   //printf("8: %lf\n", (float(t2 - t1)) / 1000);

   ///conv1
   
   
     Fdot4<<<34848,1024>>>(conv1_result_g, conv1_input, conv1_filter, LR);
     cudaDeviceSynchronize();

   

   for(int c=0; c<256; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv5_result_g[20*256*i+256*j+c];
         }
      }
      conv5_bias[c] -= tmp*LR;
   }

   for(int c=0; c<384; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv4_result_g[20*384*i+384*j+c];
         }
      }
      conv4_bias[c] -= tmp*LR;
   }


   for(int c=0; c<384; c++){
      tmp=0.0;
      for(int i=0; i<20; i++){
         for(int j=0; j<20; j++){
            tmp += conv3_result_g[20*384*i+384*j+c];
         }
      }
      conv3_bias[c] -= tmp*LR;
   } 

   for(int c=0; c<256; c++){
      tmp=0.0;
      for(int i=0; i<40; i++){
         for(int j=0; j<40; j++){
            tmp += conv2_result_g[40*256*i+256*j+c];
         }
      }
      conv2_bias[c] -= tmp*LR;
   }  
   for(int c=0; c<96; c++){
      tmp=0.0;
      for(int i=0; i<80; i++){
         for(int j=0; j<80; j++){
            tmp += conv1_result_g[81*96*i+96*j+c];
         }
      }
      conv1_bias[c] -= tmp*LR;
   }
   

    t2 = clock();
   //printf("9: %lf\n", (float(t2 - t1)) / 1000);
   
}


int main() {
   

   cudaMallocManaged(&conv1_input, sizeof(float)*336*336*3);
   cudaMallocManaged(&conv1_filter, sizeof(float)*11*11*3*96);
   cudaMallocManaged(&conv1_bias, sizeof(float)*96);
   cudaMallocManaged(&conv1_result, sizeof(float)*83*83*96);

   cudaMallocManaged(&conv2_input, sizeof(float)*40*40*96);
   cudaMallocManaged(&conv2_filter, sizeof(float)*5*5*96*256);
   cudaMallocManaged(&conv2_bias, sizeof(float)*256);
   cudaMallocManaged(&conv2_result, sizeof(float)*42*42*256); //pooling에서 오류 대비

   cudaMallocManaged(&conv3_input, sizeof(float)*20*20*256);
   cudaMallocManaged(&conv3_filter, sizeof(float)*3*3*256*384);
   cudaMallocManaged(&conv3_bias, sizeof(float)*384);
   cudaMallocManaged(&conv3_result, sizeof(float)*20*20*384);
   //no pooling
   //float conv4_input, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv4_filter, sizeof(float)*3*3*384*384);
   cudaMallocManaged(&conv4_bias, sizeof(float)*384);
   cudaMallocManaged(&conv4_result, sizeof(float)*20*20*384);
   //no pooling
   //float conv5_input, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv5_filter, sizeof(float)*3*3*384*256);
   cudaMallocManaged(&conv5_bias, sizeof(float)*256);
   cudaMallocManaged(&conv5_result, sizeof(float)*22*22*256);

   cudaMallocManaged(&conv6_input, sizeof(float)*10*10*256);
   cudaMallocManaged(&conv6_filter, sizeof(float)*10*10*256*4096);
   cudaMallocManaged(&conv6_bias, sizeof(float)*4096);

   cudaMallocManaged(&fc1, sizeof(float)*4096);
   cudaMallocManaged(&w1, sizeof(float)*4096*4096);
   cudaMallocManaged(&fc1_bias, sizeof(float)*4096);

   cudaMallocManaged(&fc2, sizeof(float)*4096);
   cudaMallocManaged(&w2, sizeof(float)*4096*6);
   cudaMallocManaged(&fc2_bias, sizeof(float)*6);

   cudaMallocManaged(&fc3, sizeof(float)*6);

   cudaMallocManaged(&fc3_g, sizeof(float)*6);
   cudaMallocManaged(&fc2_g, sizeof(float)*4096);
   cudaMallocManaged(&fc1_g, sizeof(float)*4096);
   cudaMallocManaged(&conv6_input_g, sizeof(float)*10*10*256);

   cudaMallocManaged(&conv5_result_g, sizeof(float)*20*20*256);
   cudaMallocManaged(&conv5_input_g, sizeof(float)*20*20*384);
   //float conv5_input_g[20*20*384];

   cudaMallocManaged(&conv4_result_g, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv4_input_g, sizeof(float)*20*20*384);
   //float conv4_input_g[20*20*384];

   cudaMallocManaged(&conv3_result_g, sizeof(float)*20*20*384);
   cudaMallocManaged(&conv3_input_g, sizeof(float)*20*20*256);

   cudaMallocManaged(&conv2_result_g, sizeof(float)*40*40*256);
   cudaMallocManaged(&conv2_input_g, sizeof(float)*40*40*96);

   cudaMallocManaged(&conv1_result_g, sizeof(float)*81*81*96);


   int num_testset=0;
   int randnum;
   pair <int, int> testset[2*CAT1_test+2*CAT2_test+2*CAT3_test+2*CAT4_test+2*CAT5_test+CAT6_test+10];
   clock_t t1,t2,start;
   t1=clock();
   start=clock();
   //fp=fopen("result.txt","w");
   srand(time(NULL));
   //printf("init\n");
   initdata(1);
   t2=clock();
   //printf("%lf\n",((float)(t2-t1))/1000);
   t1=clock();

   for(int i=1; i<=2*CAT1_test; i++){
      testset[num_testset++]=make_pair(1, i);
   }
   for(int i=1; i<=2*CAT2_test; i++){
      testset[num_testset++]=make_pair(2, i);
   }
   for(int i=1; i<=2*CAT3_test; i++){
      testset[num_testset++]=make_pair(3, i);
   }
   for(int i=1; i<=2*CAT4_test; i++){
      testset[num_testset++]=make_pair(4, i);
   }
   for(int i=1; i<=2*CAT5_test; i++){
      testset[num_testset++]=make_pair(5, i);
   }
   for(int i=1; i<=CAT6_test; i++){
      testset[num_testset++]=make_pair(6, i);
   }

   //데이터셋 섞기
   t1=clock();

   //데이터셋 학습
   for(int i=0; i<epoch; i++){
      if(i!=0 && i%(epoch/10)==0) LR-=0.000001;
      randnum=i%6+1;
      if(randnum==3) {
         continue;
      }
      //printf("\n%d try",i);
      //t1=clock();
      //printf("\ngetimage\n");
      getimage(randnum,rand()%datasize(randnum)+1);
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
   
      //t1=clock();
      //printf("\nforward\n");
      forward();
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
      
      //printf("backpropagation\n");
      //t1=clock();
      backpropagation(randnum-1);
      t2=clock();
      //printf("%lf\n",((float)(t2-t1))/1000);
      
   }
      
   t2=clock();
   printf("%f\n\n",((float)(t2-t1))/1000000.0);

   for(int i=0; i<num_testset; i++){
      printf("\n[CAT%d %d]",testset[i].first,testset[i].second);
     gettestimage(testset[i].first,testset[i].second);
      forward();
      if(testset[i].second>20) checkanswer1(testset[i].first-1);
     else checkanswer(testset[i].first-1);
   }
   printf("\nRESULT\n\n");
   printf("TOT1: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4]+testresult[5])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test+CAT6_test));
   printf("TOT2: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)testresult[0])/CAT1_test);
   printf("CAT2: %f\n",((float)testresult[1])/CAT2_test);
   //printf("CAT3: %f\n",((float)testresult[2])/CAT3_test);
   printf("CAT4: %f\n",((float)testresult[3])/CAT4_test);
   printf("CAT5: %f\n",((float)testresult[4])/CAT5_test);
   printf("CAT6: %f\n\n",((float)testresult[5])/CAT6_test);

   if(catresultcat[0]+catresultcat[1]+catresultcat[3]+catresultcat[4]!=0) printf("TOT3: %f\n\n", ((float)testresult[0]+testresult[1]+testresult[3]+testresult[4])/(catresultcat[0]+catresultcat[1]+catresultcat[3]+catresultcat[4]));
   if(catresultcat[0]!=0) printf("CAT1: %f\n",((float)testresult[0])/catresultcat[0]);
   if(catresultcat[1]!=0) printf("CAT2: %f\n",((float)testresult[1])/catresultcat[1]);
   //if(catresultcat[2]!=0) printf("CAT3: %f\n",((float)testresult[2])/catresultcat[2]);
   if(catresultcat[3]!=0) printf("CAT4: %f\n",((float)testresult[3])/catresultcat[3]);
   if(catresultcat[4]!=0) printf("CAT5: %f\n\n",((float)testresult[4])/catresultcat[4]);

   printf("TOT4: %f\n\n", ((float)onlycat[0]+onlycat[1]+onlycat[3]+onlycat[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)onlycat[0])/CAT1_test);
   printf("CAT2: %f\n",((float)onlycat[1])/CAT2_test);
   //printf("CAT3: %f\n",((float)onlycat[2])/CAT3_test);
   printf("CAT4: %f\n",((float)onlycat[3])/CAT4_test);
   printf("CAT5: %f\n",((float)onlycat[4])/CAT5_test);

      printf("\n\n\nRESULT2\n\n");
   printf("TOT2: %f\n\n", ((float)testresult1[0]+testresult1[1]+testresult1[3]+testresult1[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)testresult1[0])/CAT1_test);
   printf("CAT2: %f\n",((float)testresult1[1])/CAT2_test);
   printf("CAT4: %f\n",((float)testresult1[3])/CAT4_test);
   printf("CAT5: %f\n\n",((float)testresult1[4])/CAT5_test);

   if(catresultcat1[0]+catresultcat1[1]+catresultcat1[3]+catresultcat1[4]!=0) printf("TOT3: %f\n\n", ((float)testresult1[0]+testresult1[1]+testresult1[3]+testresult1[4])/(catresultcat1[0]+catresultcat1[1]+catresultcat1[3]+catresultcat1[4]));
   if(catresultcat1[0]!=0) printf("CAT1: %f\n",((float)testresult1[0])/catresultcat1[0]);
   if(catresultcat1[1]!=0) printf("CAT2: %f\n",((float)testresult1[1])/catresultcat1[1]);
   if(catresultcat1[3]!=0) printf("CAT4: %f\n",((float)testresult1[3])/catresultcat1[3]);
   if(catresultcat1[4]!=0) printf("CAT5: %f\n\n",((float)testresult1[4])/catresultcat1[4]);

   printf("TOT4: %f\n\n", ((float)onlycat1[0]+onlycat1[1]+onlycat1[3]+onlycat1[4])/(CAT1_test+CAT2_test+CAT4_test+CAT5_test));
   printf("CAT1: %f\n",((float)onlycat1[0])/CAT1_test);
   printf("CAT2: %f\n",((float)onlycat1[1])/CAT2_test);
   printf("CAT4: %f\n",((float)onlycat1[3])/CAT4_test);
   printf("CAT5: %f\n",((float)onlycat1[4])/CAT5_test);





      //printf("%f\n",((float)(t2-start))/1000000);
}

8141.892096


[CAT1 1]
0.244649 0.159347 0.000000 0.203688 0.224991 0.167325 
answer: 0
result: 0

[CAT1 2]
0.479379 0.038065 0.000000 0.043577 0.343137 0.095842 
answer: 0
result: 0

[CAT1 3]
0.374391 0.122740 0.000000 0.081201 0.243894 0.177773 
answer: 0
result: 0

[CAT1 4]
0.374787 0.083350 0.000000 0.060358 0.319870 0.161635 
answer: 0
result: 0

[CAT1 5]
0.429407 0.047497 0.000000 0.032302 0.376967 0.113827 
answer: 0
result: 0

[CAT1 6]
0.390103 0.089253 0.000000 0.056985 0.290576 0.173082 
answer: 0
result: 0

[CAT1 7]
0.117080 0.266035 0.000000 0.105862 0.152412 0.358612 
answer: 0
result: 5

[CAT1 8]
0.371778 0.085348 0.000000 0.122532 0.369733 0.050609 
answer: 0
result: 0

[CAT1 9]
0.020838 0.430305 0.000000 0.479198 0.018197 0.051462 
answer: 0
result: 3

[CAT1 10]
0.177282 0.263352 0.000000 0.090312 0.253158 0.215896 
answer: 0
result: 1

[CAT1 11]
0.049845 0.303237 0.000000 0.508814 0.067951 0.070153 
answer: 0
result: 3

[CAT1 12]
0.295743 0.161728 0.000000 0.223789 0.2